In [1]:
_RIOT_API_KEY='RGAPI-f4f1a637-f067-477c-b698-1183d53de582'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

In [3]:
match_list=help.get_matchlist_by_summoner_name('souporsecret', lol_watcher)
len(match_list)
#arena_match = lol_watcher.match.by_id('na1', match_list[0])
#flex_match = lol_watcher.match.by_id('na1', match_list[4])


933

In [9]:
match = lol_watcher.match.by_id('na1', match_list[0])
item_dict = constants.get_items(lol_watcher)
spell_dict = constants.get_spells(lol_watcher)
queues_dict = constants.get_queues()
match_info = match['info']

In [4]:
match_details = help.slurp_match_details(lol_watcher, match_list[0])
match_details

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,teamName,item1Name,item2Name,item3Name,item4Name,item5Name,item6Name,summoner1Name,summoner2Name,queueName
0,0,0,2,0,0,0,0,12500,13,22,...,Red,Guardian's Hammer,Blade of The Ruined King,Wit's End,Titanic Hydra,NaN,Arcane Sweeper,Flee,Flash,Arena
1,0,0,3,0,0,0,0,12500,13,33,...,Red,Guardian's Amulet,"Jak'Sho, The Protean",Spirit Visage,Force of Nature,NaN,Arcane Sweeper,Flee,Flash,Arena
2,0,0,4,0,0,0,0,12500,13,74,...,Red,Zhonya's Hourglass,Luden's Tempest,Rabadon's Deathcap,Sorcerer's Shoes,NaN,Arcane Sweeper,Flee,Flash,Arena
3,0,0,1,0,0,0,0,12500,13,35,...,Red,Guardian's Orb,Liandry's Anguish,Rylai's Crystal Scepter,Lich Bane,NaN,Arcane Sweeper,Flee,Flash,Arena
4,0,0,10,0,0,0,0,17000,18,61,...,Blue,Lich Bane,Luden's Tempest,Seraph's Embrace,Rabadon's Deathcap,Night Harvester,Arcane Sweeper,Flee,Flash,Arena
5,0,0,8,0,0,0,1,17000,18,22,...,Blue,Bloodthirster,Wit's End,Immortal Shieldbow,Rapid Firecannon,Mercury's Treads,Arcane Sweeper,Flee,Flash,Arena
6,0,0,6,0,0,0,0,17043,18,74,...,Blue,Morellonomicon,Luden's Tempest,Rite Of Ruin,Demonic Embrace,Shadowflame,Arcane Sweeper,Flee,Flash,Arena
7,0,0,4,0,0,0,0,17043,18,55,...,Blue,Atma's Reckoning,Hextech Gunblade,Riftmaker,Rabadon's Deathcap,Rite Of Ruin,Arcane Sweeper,Flee,Flash,Arena


In [45]:
match_details['puuid']

0    sURtv-5Qj5TWR0yuFiPU0i9ydcn-XHOMdmPrz18zyoUn5A...
1    ngIV8Vc9gyLaCa5ECbUQEJEfFGLTYca9vtXfGIL1u9qQ3T...
2    WI-mDABEZqHGDONzco_ilgzV-e4xPg7coicgKM69nfd7-s...
3    hyk4wZIriTJYUefDEmXIZcGBx2A2GYd0WMiSsZvLPC_X1K...
4    NN58S0GXj8u8tFon2r4pQdxb4ibChOtEKxTHqt2TSqV5uE...
5    _oEOnUA4Ni8R7UvmXcCk9c1ejjmjvQZGaHSIgelRZ_b2Wv...
6    ny57B9blo38O9ToaPM3M2gFaLCBnok18GjQQwf4PqZWIyb...
7    StSWemuVdV8j-y70pVC08LhERr6erPC7FWrZBysSRbTYOC...
Name: puuid, dtype: object

In [4]:
match_list[0]

'NA1_4721011388'

In [6]:
# df.to_csv('teehee.csv')


match time:  2023-07-17 20:53:05
match duration: 2220 seconds


# Match Timeline Work

In [10]:
timeline = lol_watcher.match.timeline_by_match('na1', match_list[2])
match_timeline = lol_watcher.match.timeline_by_match('na1', match_list[2])['info']


In [70]:
l = len(match_timeline['frames'])
data_fields = ['participantId','totalGold','level','xp','minionsKilled','jungleMinionsKilled','totalDamageDoneToChampions','posX','posY']
col_names = ['minute'] + data_fields
golddf = pd.DataFrame(columns = col_names)
list_of_lists_timeline = []
for i in range(l): #loops thru minutes
    for j in match_timeline['frames'][i]['participantFrames']:#loops thru champions
        champ_data=[i,  match_timeline['frames'][i]['participantFrames'][j][data_fields[0]],
                        match_timeline['frames'][i]['participantFrames'][j][data_fields[1]],
                        match_timeline['frames'][i]['participantFrames'][j][data_fields[2]],
                        match_timeline['frames'][i]['participantFrames'][j][data_fields[3]],
                        match_timeline['frames'][i]['participantFrames'][j][data_fields[4]],
                        match_timeline['frames'][i]['participantFrames'][j][data_fields[5]],
                        match_timeline['frames'][i]['participantFrames'][j]['damageStats'][data_fields[6]],
                        match_timeline['frames'][i]['participantFrames'][j]['position']['x'],
                        match_timeline['frames'][i]['participantFrames'][j]['position']['y']]
        list_of_lists_timeline.append(champ_data)

match_timeline_df = pd.DataFrame(list_of_lists_timeline, columns = col_names)
display(match_timeline_df)


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalDamageDoneToChampions,posX,posY
0,0,1,500,1,0,0,0,0,554,581
1,0,2,500,1,0,0,0,0,593,464
2,0,3,500,1,0,0,0,0,557,345
3,0,4,500,1,0,0,0,0,458,271
4,0,5,500,1,0,0,0,0,335,269
...,...,...,...,...,...,...,...,...,...,...
275,27,6,6427,12,9206,107,4,11925,13004,12926
276,27,7,9117,13,10008,7,122,12393,13096,13642
277,27,8,11072,14,11586,153,0,22434,13664,12710
278,27,9,9866,13,10309,175,0,20933,13167,13779
